In [1]:
from matplotlib import pyplot as plt
import numpy as np
import sys, os
sys.path.insert(0, os.path.dirname(os.path.abspath("__file__")))
from case_studies import *

import matplotlib
matplotlib.use('agg')

In [2]:
def wolfe_search(f, df, x, p, alpha0, c1, c2):
    """
    Implements the Wolfe line-search algorithm (Algorithm 8) which returns a
    step length fulfilling the strong Wolfe conditions.

    Arguments:
        f      : objective function f(x)
        df     : gradient function df(x)
        x      : current iterate
        p      : search direction
        alpha0 : initial upper bracket / step-length guess
        c1, c2 : Wolfe condition parameters (0 < c1 < c2 < 1)

    Returns:
        alpha    : step length satisfying the strong Wolfe conditions
        brackets : list of [l, u] brackets recorded at each iteration
    """
    # Scalar wrappers: g(a) = f(x + a*p),  g'(a) = ∇f(x+a*p)ᵀp
    def g(a):
        return f(x + a * p)

    def dg(a):
        return np.inner(df(x + a * p), p)

    l = 0
    u = alpha0
    brackets = [[l, u]]   # record initial bracket (iteration 1)

    # ── Phase 1: expansion ────────────────────────────────────────────
    while True:
        if g(u) > g(0) + c1 * u * dg(0) or g(u) > g(l):
            break
        if abs(dg(u)) < c2 * abs(dg(0)):
            return u, brackets
        if dg(u) > 0:
            break
        else:
            u = u * 2
            brackets.append([l, u])

    # ── Phase 2: bisection ────────────────────────────────────────────
    while True:
        brackets.append([l, u])
        a = (l + u) / 2
        if g(a) > g(0) + c1 * a * dg(0) or g(a) > g(l):
            u = a
        else:
            if abs(dg(a)) < c2 * abs(dg(0)):
                return a, brackets
            if dg(a) < 0:
                l = a
            else:
                u = a

In [ ]:
# ── Helpers ────────────────────────────────────────────────────────────
def check_wolfe(f, df, x, p, alpha, c1, c2):
    """Return (armijo_ok, curvature_ok)."""
    phi0  = f(x)
    dphi0 = np.inner(df(x), p)
    phi_a = f(x + alpha * p)
    dphi_a = np.inner(df(x + alpha * p), p)
    armijo    = phi_a <= phi0 + c1 * alpha * dphi0
    curvature = abs(dphi_a) <= c2 * abs(dphi0)
    return armijo, curvature

def classify_path(brackets):
    """Rough path classifier based on bracket history."""
    # Phase 1 steps: upper bracket grows; Phase 2 steps: after first shrink
    expansion_steps = sum(1 for i in range(1, len(brackets))
                          if brackets[i][1] > brackets[i-1][1])
    return len(brackets), expansion_steps

c1, c2 = 1e-3, 0.9

test_cases = [
    # (label, f, df, x, p_fn, alpha0)
    # Case A: well-scaled quadratic, large alpha0 → Armijo violation → phase 2
    ("f1 large alpha0",  f1, df1,
     0.5 * np.ones(4), lambda x: -df1(x), 10.0),

    # Case B: f3, tiny alpha0 → expansion in phase 1
    ("f3 tiny alpha0",   f3, df3,
     0.1 * np.ones(3), lambda x: -df3(x), 1e-8),

    # Case C: f3, reasonable alpha0 → typically satisfies Wolfe in phase 1
    ("f3 good alpha0",   f3, df3,
     0.1 * np.ones(3), lambda x: -df3(x), 0.02),

    # Case D: Rosenbrock, needs bisection refinement
    ("f2 rosenbrock",    f2, df2,
     np.array([-1.2, 1.0]), lambda x: -df2(x), 1.0),
]

print(f"{'Case':<22} {'alpha':>12}  {'iters':>6}  {'expand':>7}  {'Armijo':>7}  {'Curv':>7}  {'PASS':>5}")
print("-" * 75)
all_passed = True
for label, f, df, x0, p_fn, alpha0 in test_cases:
    p = p_fn(x0)
    alpha, brackets = wolfe_search(f, df, x0, p, alpha0, c1, c2)
    armijo, curv = check_wolfe(f, df, x0, p, alpha, c1, c2)
    iters, exp = classify_path(brackets)
    ok = armijo and curv
    all_passed = all_passed and ok
    print(f"{label:<22} {alpha:>12.4e}  {iters:>6}  {exp:>7}  {str(armijo):>7}  {str(curv):>7}  {'✓' if ok else '✗':>5}")

print()
print("All tests passed!" if all_passed else "SOME TESTS FAILED.")